# Topics


In [15]:
!pip install z3-solver

In [16]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from z3 import *
from tqdm import tqdm

In [17]:
sys.setrecursionlimit(1500)

In [18]:
with open("13-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'Button A: X+92, Y+17\nButton B: X+27, Y+61\nPrize: X=9152, Y=6172\n\nButton A: X+12, Y+61\nButton B: X+41, Y+12\nPrize: X=8771, Y=12729\n\nButton A: X+71, Y+11\nButton B: X+14, Y+54\nPrize: X=7189, Y=5889\n\nButton A: X+22, Y+36\nButton B: X+49, Y+21\nPrize: X=8122, Y=6806\n\nButton A: X+44, Y+88\nButton B: X+72, Y+36\nPrize: X=4972, Y=4004\n\nButton A: X+11, Y+57\nButton B: X+50, Y+18\nPrize: X=3024, Y=17456\n\nButton A: X+88, Y+91\nButton B: X+80, Y+13\nPrize: X=12096, Y=8255\n\nButton A: X+22, Y+50\nButton B: X+48, Y+28\nPrize: X=17238, Y=1530\n\nButton A: X+67, Y+19\nButton B: X+25, Y+70\nPrize: X=15607, Y=12559\n\nButton A: X+20, Y+58\nButton B: X+67, Y+22\nPrize: X=13571, Y=12102\n\nButton A: X+36, Y+74\nButton B: X+82, Y+39\nPrize: X=6484, Y=8146\n\nButton A: X+15, Y+73\nButton B: X+89, Y+17\nPrize: X=9054, Y=4114\n\nButton A: X+40, Y+23\nButton B: X+12, Y+41\nPrize: X=10476, Y=1521\n\nButton A: X+61, Y+30\nButton B: X+43, Y+71\nPrize: X=5659, Y=5525\n\nButton A: X+56, Y+20\nBu

In [19]:
test_data_raw = r"""Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""


Claw = namedtuple("Claw", ["XA", "YA", "XB", "YB", "X", "Y"])


def preprocess_data (data):
    
    def prep_setup(a):
        
        XA, YA = re.findall(r'\d+', a[0])
        XB, YB = re.findall(r'\d+', a[1])
        X, Y = re.findall(r'\d+', a[2])
        # YA = int(a[0].split("+")[-1])
        # XA = int(a[0].split("+")[-2].split(",")[0])
        # YB = int(a[1].split("+")[-1])
        # XB = int(a[1].split("+")[-2].split(",")[0])
        
        return Claw(XA=int(XA), YA=int(YA), XB=int(XB), YB=int(YB), X=int(X), Y=int(Y))
    
    rows = [prep_setup(row.split("\n")) for row in data.split("\n\n")]
    return rows



test_data = preprocess_data(test_data_raw)

display(test_data)


[Claw(XA=94, YA=34, XB=22, YB=67, X=8400, Y=5400),
 Claw(XA=26, YA=66, XB=67, YB=21, X=12748, Y=12176),
 Claw(XA=17, YA=86, XB=84, YB=37, X=7870, Y=6450),
 Claw(XA=69, YA=23, XB=27, YB=71, X=18641, Y=10279)]

In [20]:
full_data = preprocess_data(data_raw)
full_data

[Claw(XA=92, YA=17, XB=27, YB=61, X=9152, Y=6172),
 Claw(XA=12, YA=61, XB=41, YB=12, X=8771, Y=12729),
 Claw(XA=71, YA=11, XB=14, YB=54, X=7189, Y=5889),
 Claw(XA=22, YA=36, XB=49, YB=21, X=8122, Y=6806),
 Claw(XA=44, YA=88, XB=72, YB=36, X=4972, Y=4004),
 Claw(XA=11, YA=57, XB=50, YB=18, X=3024, Y=17456),
 Claw(XA=88, YA=91, XB=80, YB=13, X=12096, Y=8255),
 Claw(XA=22, YA=50, XB=48, YB=28, X=17238, Y=1530),
 Claw(XA=67, YA=19, XB=25, YB=70, X=15607, Y=12559),
 Claw(XA=20, YA=58, XB=67, YB=22, X=13571, Y=12102),
 Claw(XA=36, YA=74, XB=82, YB=39, X=6484, Y=8146),
 Claw(XA=15, YA=73, XB=89, YB=17, X=9054, Y=4114),
 Claw(XA=40, YA=23, XB=12, YB=41, X=10476, Y=1521),
 Claw(XA=61, YA=30, XB=43, YB=71, X=5659, Y=5525),
 Claw(XA=56, YA=20, XB=36, YB=75, X=15692, Y=11255),
 Claw(XA=12, YA=54, XB=28, YB=27, X=540, Y=945),
 Claw(XA=62, YA=18, XB=47, YB=71, X=8201, Y=6109),
 Claw(XA=55, YA=90, XB=95, YB=35, X=7345, Y=9610),
 Claw(XA=92, YA=77, XB=26, YB=76, X=9400, Y=9820),
 Claw(XA=76, YA=75, XB

In [21]:

def solution (data, verbose=False):
    
    total = 0
    for claw in data:
        # print(claw)
        for b in range(101):
            a1 = (claw.Y - claw.YB*b) / claw.YA
            a2 = (claw.X - claw.XB*b) / claw.XA
            if a1 == a2 and a1.is_integer():
                # print(a1, b)
                total += 3*int(a1) + b
                break
        
    return total



sol = solution(test_data, verbose=True)
print(sol)



480


In [22]:
sol = solution(full_data)
print(sol)

33209


# Part 2

In [67]:
# Doesn't work because of rounding errors 
# Used z3 solution

def solution2 (data, verbose=False):
    
    ds = namedtuple("Diophine_solution", ("A", "bg", "B", "ag", "upper", "lower", "valid")) 
    
    def gcdExtended(a, b): 
        # Base Case 
        if a == 0 : 
            return b,0,1
                
        gcd,x1,y1 = gcdExtended(b%a, a) 
        # Update x and y using results of recursive 
        x = y1 - (b//a) * x1 
        y = x1 
        
        return gcd,x,y 
    
    
    solutions = pd.read_csv("solutions.csv")
    
    total = 0
    for idx, claw in enumerate(data):
        
        # Solve equiation of the form ax+by=c with integer solutions x,y
        def solve_diophine(a,b,c):
            g = np.gcd(a, b)
            if (c / g).is_integer():
                gcd, x, y = gcdExtended(a/g, b/g)
                # x * a/b + y* b/g = 1
                # Solutions to equation: A = c/g * x , B = c/g * y
                A = c/g * x
                B = c/g * y
                
                # All solutions (x2, y2) for whole number t
                #   x2 = A + (b/g) * t
                #   y2 = B - (a/g) * t
                
                #  if x2> 0 and y2>0:  
                #     B * g/a > t > -A*g/b
                upper = int(np.floor(B * g/a))
                lower = int(np.ceil(-A*g/b))
                
                if upper < lower:
                    print("No positive solution")
                    
                else:
                    
                    return ds(A=A, bg=b/g, B=B, ag=a/g, upper=upper, lower=lower, valid=True)
                    # t_values = list(range(lower, upper+1))
                    # solutions = [(int(A + (b/g) * t), int(B - (a/g) * t)) for t in t_values]
                    # print("t can be following values:", t_values)
                    # print(f"{len(solutions)} solutions:", solutions)
                    # return solutions
            
            return ds(A=-1, bg=-1, B=-1, ag=-1, upper=-1, lower=-1, valid=False)

        s1 = solve_diophine(claw.XA, claw.XB, claw.X +10000000000000 )
        s2 = solve_diophine(claw.YA, claw.YB, claw.Y + 10000000000000 )

        solution_exists = False
        if  s1.valid and s2.valid:
            necessary = (s2.ag / s1.ag - s2.bg / s1.bg)
            if necessary != 0:
                
                t2 = ((s2.A - s1.A)/s1.bg - (s1.B - s2.B)/s1.ag ) / necessary
                t1 = (s2.A - s1.A +s2.bg*t2)/s1.bg
                t1_int, t2_int = np.rint([t1, t2])
                eps = 0.07
                if abs(t1_int - t1) > eps or  abs(t2_int - t2) > eps:
                    
                    if idx in [12, 228, 242]:
                        print(t1, t2)
                        print("No analytical solution")
                else: 

                    s1_sol = (int(s1.A + s1.bg * t1_int), int(s1.B - s1.ag * t1_int))
                    s2_sol = (int(s2.A + s2.bg * t2_int), int(s2.B - s2.ag * t2_int))

                    total += s1_sol[0]*3 + s1_sol[1]
                    solution_exists = True
        else:
            pass
        
        real_solution = (solutions.iloc[idx]["Sat"] == "sat")
        if  (real_solution ^ solution_exists) or  (real_solution and solution_exists and  ((int(solutions.iloc[idx]["A"]) - s1_sol[0] != 0) or (int(solutions.iloc[idx]["B"]-s1_sol[1]) != 0))):
            print(idx, claw)
            if real_solution:
                print("Real Solutions: ", int(solutions.iloc[idx]["A"]), int(solutions.iloc[idx]["B"]))
            else:
                print("Real Solutions: Unsat")
            
            if solution_exists:
                print("Analytical solution:", s1_sol)
                if real_solution:
                    print("Difference: ", int(solutions.iloc[idx]["A"]) - s1_sol[0], int(solutions.iloc[idx]["B"]-s1_sol[1]) )
            else:
                print("No analytical solution")  
            
            
            print("\n")
        
        
    
    return total




sol = solution2(test_data, verbose=True)
print("Solution", sol)


    

1 Claw(XA=26, YA=66, XB=67, YB=21, X=12748, Y=12176)
Real Solutions:  123037760041 207891387519
Analytical solution: (118679050709, 103199174542)
Difference:  4358709332 104692212977


2 Claw(XA=17, YA=86, XB=84, YB=37, X=7870, Y=6450)
Real Solutions:  108695652257 163043478353
No analytical solution


3 Claw(XA=69, YA=23, XB=27, YB=71, X=18641, Y=10279)
Real Solutions:  215053763566 107526881830
Analytical solution: (102851800151, 107526881786)
Difference:  112201963415 44


Solution 480 875318608908


In [68]:
sol = solution2(full_data)
print("Solution", sol)

102 Claw(XA=20, YA=83, XB=29, YB=15, X=1607, Y=2139)
Real Solutions: Unsat
Analytical solution: (66445182741, 299003322303)


Solution 83102355665474 83600694536000
Diff: 498338870526
